In [1]:
import pandas as pd
import re
import json
import itertools
import math
import numpy as np

In [2]:
#read dict .json matches file
m_dir = "match_result/filtered_hit.json"
match_dict = json.load(open(m_dir))

In [3]:
#read excel file containing the metabolite abundance
s_dir = "raw/ww_ut_metabolomics_duplicateremoved.xlsx"
rawdf = pd.read_excel(s_dir, sheet_name=2)
finaldf = rawdf[['Mass', 'CIPS 1A', 'CIPS 2A', 'CIPS 3A',
                'Ciscar 1A', 'Ciscar 2A', 'Ciscar 3A',
                'General 1A', 'General 2A', 'General 3A',
                'Pesset 1A', 'Pesset 2A', 'Pesset 3A']]

In [ ]:
#create dataframe with columns containing found_mass, location, time, and value

finaldf_1 = pd.DataFrame(columns = ['found_mass', 'location', 'time', 'auc', 'tic_auc', 'log2_auc'])
time_key = {'1A':'Morning', '2A':'Afternoon', '3A':'Evening'}

for x in range(0, len(finaldf)):
    found_mass = finaldf.iloc[x,0]
    
    tmpdict = {}
    
    for y in finaldf.loc[:, 'CIPS 1A':'Pesset 3A'].columns:
        
        tmpdict['found_mass'] = found_mass
        tmpdict['auc'] = finaldf.loc[x,y]
        tmpdict['log2_auc'] = np.log(tmpdict['auc'])
        
        tic = sum(finaldf.loc[:, y])
        tmpdict['tic_auc'] = finaldf.loc[x,y]/tic
        
        tmpdict['time'] = time_key[y.split()[1]]
        tmpdict['location'] = y.split()[0]
            
        finaldf_1 = pd.concat([finaldf_1, pd.DataFrame([tmpdict])], ignore_index=True)

finaldf_1

In [ ]:
#create dataframe with columns containing list of potential matches
finaldict = {}
for x in range(0, len(finaldf)):
    found_mass = finaldf.iloc[x,0]
    
    matches = match_dict[str(found_mass)]
    
    matchlist = []
    
    for z in matches.keys():
        matchlist.append(matches[z]['name'])
        
    finaldict[found_mass] = str(matchlist)
    
dfd = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in finaldict.items() ])).T

dfd.to_csv("name.csv")

In [4]:
#create dataframe with columns containing found_mass, location, time, and value

finaldf_1 = pd.DataFrame(columns = ['found_mass', 'location', 'time', 'auc', 'log10_auc', 'chem_name', 'chem_tax_king', "chem_tax_supclass",
                                   "chem_tax_class", "chem_tax_subclass", "chem_source", "chem_ont_class", "chem_ont_subclass", 
                                    "chem_ont_reldiet", 'chem_ont_reldrug',"chem_ont_reldisease"])
time_key = {'1A':'Morning', '2A':'Afternoon', '3A':'Evening'}

for x in range(0, len(finaldf)):
    found_mass = finaldf.iloc[x,0]
    
    tmpdict = {}
    
    for y in finaldf.loc[:, 'CIPS 1A':'Pesset 3A'].columns:
        
        matches = match_dict[str(found_mass)]
        
        for z in matches.keys():
        
            tmpdict['found_mass'] = found_mass
            tmpdict['auc'] = finaldf.loc[x,y]
            tmpdict['log10_auc'] = np.log10(tmpdict['auc'])
            tic = sum(finaldf.loc[:, y])
            tmpdict['tic_auc'] = finaldf.loc[x,y]/tic
            
            tmpdict['time'] = time_key[y.split()[1]]
            tmpdict['location'] = y.split()[0]
            
            tmpdict['chem_name'] = matches[z]['name']
            
            tmpdict['chem_tax_king'] = matches[z]['taxonomy']['kingdom']
            tmpdict['chem_tax_supclass'] = matches[z]['taxonomy']['superclass']
            tmpdict['chem_tax_class'] = matches[z]['taxonomy']['class']
            tmpdict['chem_tax_subclass'] = matches[z]['taxonomy']['subclass']
            
            tmpdict['chem_source'] = matches[z]['ontology']['source']
            tmpdict['chem_ont_class'] = matches[z]['ontology']['chem_ont_class']
            tmpdict['chem_ont_subclass'] = matches[z]['ontology']['chem_ont_subclass']
            tmpdict['chem_ont_reldiet'] = matches[z]['ontology']['chem_ont_reldiet']
            tmpdict['chem_ont_reldrug'] = matches[z]['ontology']['chem_ont_reldrug']
            tmpdict['chem_ont_reldisease'] = matches[z]['ontology']['chem_ont_reldisease']
            
            if len(matches) > 1:
                tmpdict['multiple_matches'] = 'True'
            else:
                tmpdict['multiple_matches'] = 'False'
            
            finaldf_1 = pd.concat([finaldf_1, pd.DataFrame([tmpdict])], ignore_index=True)

In [ ]:
finaldf_1.to_csv('match_result/finalmatched_massonly.csv', index=False)

In [5]:
finaldf_2 = finaldf_1[finaldf_1['chem_ont_subclass'] == 'antibiotics'].reset_index()
finaldf_2.to_csv('match_result/finalmatched_antibiotics.csv', index=False)